# Business Context

Job Post Response Email generator for an individual  using groq and langchain. It allows users to input the URL of a company's careers page. The tool then extracts job listings from that page and generates personalized  email. These emails include relevant portfolio links sourced from a vector database, based on the specific job descriptions.

Imagine a scenario:

There is a job posting in dice.com. You provide the direct URL to the job post to the LLM, then this program will generate an email response to the job post highlighting on how your skill match the requirement.

*Credit: codebasics youtube channel.*

# Install langchain_groq langchain_community chromadb

1. `!`: This is used in Jupyter notebooks or other interactive environments to run shell commands directly from a code cell.

2. `pip install`: This command tells `pip` to install the specified Python packages. `pip` is a tool for installing and managing Python packages.

3. `langchain_groq`: This is one of the packages being installed. The exact functionality would depend on the package, but in general, `langchain` packages are likely related to language models and their applications.

4. `langchain_community`: Another package that’s probably related to community-driven contributions for language models.

5. `chromadb`: This could be a package for managing databases or a similar functionality, but the specific details would depend on the package itself.

In summary, this command installs three Python packages, allowing you to use their functionalities in your Python environment.

In [ ]:
!pip install langchain_groq langchain_community chromadb

### Importing Libraries

In [ ]:
import pandas as pd

In [ ]:
import uuid
import chromadb

**`langchain_groq`**: This is the name of the module or package from which we're importing. It likely contains various functionalities related to language models or natural language processing.

**`ChatGroq`**: This is the specific class or function being imported from the `langchain_groq` module.

By importing `ChatGroq`, you can create instances or use functions of the `ChatGroq` class in your Python code. This is useful for organizing code and using specific functionalities without having to import the entire module.

I am using groqcloud.com to reduce the time taken for inference. This platform uses language processing unit(lpu) that makes inference process so much faster.

In [ ]:
from langchain_groq import ChatGroq

**ChatGroq:** This is a class from the langchain_groq module, likely designed to interact with language models.

**temperature=0:** The temperature parameter controls the randomness of the model's output. A temperature of 0 means the model will produce more deterministic and focused responses.

**groq_api_key='#####################':** This is an API key used to authenticate and authorize access to the Groq service.In your project do not use this key. This key is specific to the project i have created in console.Groqcloud.com

**model_name="llama-3.1-70b-versatile":** This specifies the name of the model to be used. Here, it’s a version of the llama model.

**llm.invoke("The first person to climb Mt. Everest is ..."):** This line sends a prompt to the language model instance llm to generate a response based on the input string "The first person to climb Mt. Everest is ...".
You can see the result below under output.

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='###########################################################',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to climb Mt. Everest is ...")
print(response.content)

The first people to climb Mt. Everest were Sir Edmund Hillary from New Zealand and Tenzing Norgay, a Nepali Sherpa mountaineer, on May 29, 1953.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.dice.com/job-detail/62108b45-79d5-427b-a597-26c02963e249?searchlink=search%2F%3Fq%3Dprogrammer%2520dev%2520ops%26countryCode%3DUS%26radius%3D30%26radiusUnit%3Dmi%26page%3D1%26pageSize%3D20%26language%3Den&searchId=b608a277-04ea-4c94-8702-3274038e03f6")
page_data = loader.load().pop().page_content
print(page_data)

Devops Release Engineer(salesforce must) - V United Software Inc - Trenton, NJHomeSearchJob Details OptionsShareReportDevops Release Engineer(salesforce must)V United Software IncTrenton, NJPosted 5 days ago |  Updated 1 day agoOverviewHybrid2 DAYS ONSITE$90 - $90Accepts corp to corp applicationsContract - IndependentContract - 1 Year(s)No Travel RequiredSkillsAmazon Web ServicesGitDockerGrafanaPipeline managementPythonSalesforce.comKubernetesDevOpsdevops release engineerdevops programmeraws devopsContinuous deliveryCI/CDScriptingProgress ChefTerraformPuppetBashAnsibleVersion controlContinuous integrationGitHubIaaSJenkinsJob DetailsSkill 12 years of Genuine IT experience requiredExperience developing and managing Salesforce CI/CD pipelinesAWS DevOpsCloud Infrastructure Management (AWS)Scripting (Python, Bash, PowerShell)Version Control (Git)Performance OptimizationSecurity Best PracticesProblem-SolvingCommunication and Collaboration with TeamsMonitoring (Prometheus, Grafana)CI/CD Pipel

**Importing the PromptTemplate Class**:
This line imports the PromptTemplate class from the langchain_core.prompts module. The PromptTemplate class is used to create and manage text templates that can be filled in with specific data.

In [ ]:
from langchain_core.prompts import PromptTemplate

**Creating a Prompt Template**:
* PromptTemplate.from_template(...): This method creates a prompt template from the provided text. The template includes placeholders (e.g., {page_data}) that can be filled with actual data later.

* The template specifies that the scraped text (to be provided in {page_data}) comes from a career page of a website. The instruction to the model is to extract job postings and return them in JSON format with specific keys: role, experience, skills, and description.

In [ ]:
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

**Combining the Prompt with a Language Model**:
* This line creates a chain (chain_extract) by combining the prompt_extract with a language model (llm). The | operator is used to create a pipeline where the prompt is processed by the language model. The language model (llm) was defined previously (presumably as an instance of ChatGroq).

In [ ]:
chain_extract = prompt_extract | llm


**Invoking the Model with Data**

> chain_extract.invoke(input={'page_data': page_data}): This line invokes the chain with actual input data. It replaces the {page_data} placeholder in the prompt template with the content of the page_data variable. The model processes this input and generates a response.



In [ ]:
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Devops Release Engineer',
 'experience': '12 years of Genuine IT experience required',
 'skills': ['Amazon Web Services',
  'Git',
  'Docker',
  'Grafana',
  'Pipeline management',
  'Python',
  'Salesforce.com',
  'Kubernetes',
  'DevOps',
  'devops release engineer',
  'devops programmer',
  'aws devops',
  'Continuous delivery',
  'CI/CD',
  'Scripting',
  'Progress Chef',
  'Terraform',
  'Puppet',
  'Bash',
  'Ansible',
  'Version control',
  'Continuous integration',
  'GitHub',
  'IaaS',
  'Jenkins'],
 'description': 'As a Senior Release DevOps Engineer, you will be instrumental in managing and optimizing our CI/CD pipelines, beginning with manual coordination of GitHub code releases across Salesforce teams and transitioning to Copado for automated releases. Your skills in Copado, AWS DevOps, and relevant tools will streamline our release processes and strengthen deployment strategies. You ll work closely with development, QA, and operations teams to ensure a reliable,

In [ ]:
type(json_res)

dict

## Read the CSV to the dataframe

In [ ]:
url = 'https://raw.githubusercontent.com/codebasics/project-genai-cold-email-generator/refs/heads/main/app/resource/my_portfolio.csv'
df = pd.read_csv(url)
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


## Initialize ChormaDB and Create Collections

The code snippet initializes a ChromaDB client and accesses (or creates) a collection named "portfolio". If the collection is empty, it iterates through each row in a DataFrame df, adding documents to the collection. Each document consists of the Techstack value as the document, Links value as metadata, and a unique identifier generated for each entry.

This is a common pattern for preparing data in vector databases for machine learning or other advanced analytics.

In [ ]:
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
# JSON
job = json_res
job['skills']

['Amazon Web Services',
 'Git',
 'Docker',
 'Grafana',
 'Pipeline management',
 'Python',
 'Salesforce.com',
 'Kubernetes',
 'DevOps',
 'devops release engineer',
 'devops programmer',
 'aws devops',
 'Continuous delivery',
 'CI/CD',
 'Scripting',
 'Progress Chef',
 'Terraform',
 'Puppet',
 'Bash',
 'Ansible',
 'Version control',
 'Continuous integration',
 'GitHub',
 'IaaS',
 'Jenkins']

**Querying the Collection**<BR>
The below code performs a query on a ChromaDB collection using the skills attribute from a job dictionary, requesting the top 2 results. It then extracts the metadata from the query results and stores them in the links variable, which is then displayed.

In essence, this code is used to search for and retrieve relevant metadata associated with job skills from a vector database.

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'

In [ ]:
job

{'role': 'Devops Release Engineer',
 'experience': '12 years of Genuine IT experience required',
 'skills': ['Amazon Web Services',
  'Git',
  'Docker',
  'Grafana',
  'Pipeline management',
  'Python',
  'Salesforce.com',
  'Kubernetes',
  'DevOps',
  'devops release engineer',
  'devops programmer',
  'aws devops',
  'Continuous delivery',
  'CI/CD',
  'Scripting',
  'Progress Chef',
  'Terraform',
  'Puppet',
  'Bash',
  'Ansible',
  'Version control',
  'Continuous integration',
  'GitHub',
  'IaaS',
  'Jenkins'],
 'description': 'As a Senior Release DevOps Engineer, you will be instrumental in managing and optimizing our CI/CD pipelines, beginning with manual coordination of GitHub code releases across Salesforce teams and transitioning to Copado for automated releases. Your skills in Copado, AWS DevOps, and relevant tools will streamline our release processes and strengthen deployment strategies. You ll work closely with development, QA, and operations teams to ensure a reliable,

In [ ]:
job = json_res
job['skills']

['Amazon Web Services',
 'Git',
 'Docker',
 'Grafana',
 'Pipeline management',
 'Python',
 'Salesforce.com',
 'Kubernetes',
 'DevOps',
 'devops release engineer',
 'devops programmer',
 'aws devops',
 'Continuous delivery',
 'CI/CD',
 'Scripting',
 'Progress Chef',
 'Terraform',
 'Puppet',
 'Bash',
 'Ansible',
 'Version control',
 'Continuous integration',
 'GitHub',
 'IaaS',
 'Jenkins']

# Prompt Template

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Vijay, a Dev Ops Engineer. You have more than 12  years of experience in this field. Currently you are working at Microsoft as a Senior Dev Ops Engineer.
        Over at Microsoft you have delivered several projects in this field successfully.
        Your job is to write a cold email to the client regarding the job mentioned above describing your capability, your experience
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase my portfolio: {link_list}
        Remember you are Vijay, a Dev Ops Engineer.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )



# Email Generation

The code below creates a pipeline that combines a prompt template (prompt_email) with a language model (llm). It invokes this chain with a job description and a list of links as inputs, and then prints the generated response content, which is likely an email tailored to the job description and links.

This is a common approach for automating text generation tasks using language models and structured prompts. If you need further details or have any other questions, feel free to ask!

In [ ]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for DevOps Release Engineer Position

Dear Hiring Manager,

I am writing to express my interest in the DevOps Release Engineer position at your esteemed organization. With over 12 years of experience in the field of DevOps, I am confident that I possess the skills and expertise required to excel in this role.

As a Senior DevOps Engineer at Microsoft, I have had the opportunity to work on numerous projects, delivering high-quality results and ensuring seamless deployment of applications. My expertise in Amazon Web Services, Git, Docker, Grafana, Pipeline management, Python, and other relevant tools aligns perfectly with the requirements of this position.

I am particularly drawn to this role because of the emphasis on managing and optimizing CI/CD pipelines, which is an area I have extensive experience in. My skills in Copado, AWS DevOps, and other relevant tools will enable me to streamline release processes and strengthen deployment strategies.

In my current rol

# Insights

* With few lines of code we are able to use LLAMA Model to generate an email specific to a Job Post.
* Through this simple project we are able to see the power of AI and its capabilities.
* Get better at writing prompt templates. This helps in getting better results. Sometimes it is trial and error.
* A job seeker can leverage technology to outsmart companies that use AI to filter applications, as technology is becoming increasingly accessible to everyone.